# 计算每个学生成绩最相似的10个学生

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("./datas/student_grade/学生成绩.csv")

In [3]:
df.head(5)

姓名  Python程序设计基础  计算机导论  离散数学  数据结构  C语言程序设计  java语言程序设计  算法导论
0  小明1            54     73    98    69       76          78    67
1  小明2            51     70    83    93       93          93    96
2  小明3            84     61   100    84       60          97    71
3  小明4            71     56    57    89       52          62    93
4  小明5            78     61    54    96       83         100    56

In [4]:
df.shape

(60, 8)

## 1. 进行学生成绩的笛卡尔积

In [5]:
df["one"] = 1

In [6]:
df.head(3)

姓名  Python程序设计基础  计算机导论  离散数学  数据结构  C语言程序设计  java语言程序设计  算法导论  one
0  小明1            54     73    98    69       76          78    67    1
1  小明2            51     70    83    93       93          93    96    1
2  小明3            84     61   100    84       60          97    71    1

In [7]:
df_merge = pd.merge(left=df, right=df, left_on="one", right_on="one")

In [8]:
df_merge.head(3)

姓名_x  Python程序设计基础_x  计算机导论_x  离散数学_x  数据结构_x  C语言程序设计_x  java语言程序设计_x  \
0  小明1              54       73      98      69         76            78   
1  小明1              54       73      98      69         76            78   
2  小明1              54       73      98      69         76            78   

   算法导论_x  one 姓名_y  Python程序设计基础_y  计算机导论_y  离散数学_y  数据结构_y  C语言程序设计_y  \
0      67    1  小明1              54       73      98      69         76   
1      67    1  小明2              51       70      83      93         93   
2      67    1  小明3              84       61     100      84         60   

   java语言程序设计_y  算法导论_y  
0            78      67  
1            93      96  
2            97      71

In [9]:
df_merge.shape

(3600, 17)

## 2. 两两计算相似度

In [10]:
columns = list(df.columns)
columns.remove("姓名")
columns.remove("one")
columns

['Python程序设计基础', '计算机导论', '离散数学', '数据结构', 'C语言程序设计', 'java语言程序设计', '算法导论']

In [11]:
def sim_fun(row):
    sim_value = 0.0
    for column in columns:
        sim_value += abs(int(row[column+"_x"]) - int(row[column+"_y"]))
    return sim_value

In [12]:
df_merge["sim"] = df_merge.apply(sim_fun, axis=1)

In [13]:
df_merge.head(3)

姓名_x  Python程序设计基础_x  计算机导论_x  离散数学_x  数据结构_x  C语言程序设计_x  java语言程序设计_x  \
0  小明1              54       73      98      69         76            78   
1  小明1              54       73      98      69         76            78   
2  小明1              54       73      98      69         76            78   

   算法导论_x  one 姓名_y  Python程序设计基础_y  计算机导论_y  离散数学_y  数据结构_y  C语言程序设计_y  \
0      67    1  小明1              54       73      98      69         76   
1      67    1  小明2              51       70      83      93         93   
2      67    1  小明3              84       61     100      84         60   

   java语言程序设计_y  算法导论_y    sim  
0            78      67    0.0  
1            93      96  106.0  
2            97      71   98.0

In [14]:
df_merge = df_merge[df_merge["姓名_x"] != df_merge["姓名_y"]].copy()

In [15]:
df_merge.head(3)

姓名_x  Python程序设计基础_x  计算机导论_x  离散数学_x  数据结构_x  C语言程序设计_x  java语言程序设计_x  \
1  小明1              54       73      98      69         76            78   
2  小明1              54       73      98      69         76            78   
3  小明1              54       73      98      69         76            78   

   算法导论_x  one 姓名_y  Python程序设计基础_y  计算机导论_y  离散数学_y  数据结构_y  C语言程序设计_y  \
1      67    1  小明2              51       70      83      93         93   
2      67    1  小明3              84       61     100      84         60   
3      67    1  小明4              71       56      57      89         52   

   java语言程序设计_y  算法导论_y    sim  
1            93      96  106.0  
2            97      71   98.0  
3            62      93  161.0

## 3. 计算每个学生的TOP N的学生

In [16]:
def get_top_student(df_sub):
    df_sort = df_sub.sort_values(by="sim", ascending=False).head(10)
    names = ",".join(list(df_sort["姓名_y"]))
    sims = ",".join([str(x) for x in list(df_sort["sim"])])
    return pd.Series({"names": names, "sims": sims})

In [17]:
df_result = df_merge.groupby("姓名_x").apply(get_top_student)

In [18]:
df_result.to_excel("./datas/student_grade/相似计算结果.xlsx", index=True)